In [402]:
import pandas as pd
import numpy as np
import os

In [403]:
def extract_csv_data(csv_file, enumerate_num):
    df = pd.read_csv(f'inputs/{csv_file}')
    payments = df.shape[0] + 1
    payout_date = df['payouts.arrival_date'][0]
    payout_date = pd.to_datetime(payout_date)
    next_payout_sum = df['net_amount'].sum()

    output_str = f"""
    ID: {enumerate_num+1}
    Number of Payments: {payments}
    Date of Payout: {payout_date.strftime('%d-%b-%Y')}
    Payout Amount: £{round(next_payout_sum,2)}
    """

    print(output_str)
    return df

In [404]:
def extract_payment_amounts(df, split = 'schedule'):
    sections = ['Squirrels', 'Beavers', 'Cubs', 'Scouts']
    payment_schedules = ['Activities', 'Monthly Subscriptions']

    result_subsets = []  # List to store subsets

    df = df.groupby(['resources.description'])[['gross_amount', 'gocardless_fees', 'app_fees', 'net_amount']].sum().reset_index()

    for section in sections:
        subset_section = df[df['resources.description'].str.contains(section, case=False)].copy()

        for schedule in payment_schedules:
            subset_schedule = subset_section[subset_section['resources.description'].str.contains(schedule, case=False)].copy()
            subset_schedule['Section'] = section
            subset_schedule['Payment Schedule'] = schedule
            subset_schedule['Section Payment Schedule'] = section + ": " + schedule

            result_subsets.append(subset_schedule)

    # Concatenate all subsets into a new DataFrame
    df = pd.concat(result_subsets, ignore_index=True)
    
    # Group so all sections payments for each schedule are together
    if split == 'schedule':
        df = df.groupby(['Section', 'Payment Schedule'])[['gross_amount', 'gocardless_fees', 'app_fees', 'net_amount']].sum().reset_index()
        df = df.sort_values(by=['Section', 'Payment Schedule'])
    elif split == 'section':
        df = df.groupby(['Section'])[['gross_amount', 'gocardless_fees', 'app_fees', 'net_amount']].sum().reset_index()
        df = df.sort_values(by=['Section'])
  
    return df

In [405]:
def sort_and_order_df(df):
    
   df['total_fees'] = df['gross_amount'] - df['net_amount']

   column_order = ['Section', 'Payment Schedule', 'net_amount', 'total_fees', 'gross_amount', 'gocardless_fees','app_fees']
    
   df = df[column_order]
  
   column_sums = df.sum()
   
   total_row = pd.DataFrame([column_sums.values], columns=column_sums.index, index=['Total'])
    
   df = pd.concat([df, total_row])

   currency_columns = ['net_amount', 'total_fees', 'gross_amount', 'gocardless_fees', 'app_fees']
   df[currency_columns] = df[currency_columns].round(2)

   df.at['Total', 'Section'] = ''
   df.at['Total', 'Payment Schedule'] = ''
   df = df.style.apply(lambda x: ['background-color: #008080']*len(x), subset=pd.IndexSlice[df.index[-1], :])
   df = df.set_properties(**{'text-align': 'left'}, subset=[('Section')])
   df = df.set_properties(**{'text-align': 'left'}, subset=[('Payment Schedule')])

   return df

In [406]:
def by_schedule():
    csv_files = [f for f in os.listdir('inputs') if f.endswith(".csv")]
    for idx, csv_file in enumerate(csv_files):
        df = extract_csv_data(csv_file=csv_file, enumerate_num=idx)
        df = extract_payment_amounts(df)
        df = sort_and_order_df(df)
        display(df)
        df.to_excel(f'outputs/{idx}_output.xlsx', index=False)
by_schedule()
        


    ID: 1
    Number of Payments: 39
    Date of Payout: 12-Jan-2024
    Payout Amount: £489.68
    


,Section,Payment Schedule,net_amount,total_fees,gross_amount,gocardless_fees,app_fees
0,Beavers,Activities,43.200000,1.800000,45.000000,0.930000,0.870000
1,Beavers,Monthly Subscriptions,34.440000,1.560000,36.000000,0.870000,0.690000
2,Cubs,Monthly Subscriptions,344.400000,15.600000,360.000000,8.700000,6.900000
3,Scouts,Activities,56.160000,1.840000,58.000000,0.710000,1.130000
4,Squirrels,Monthly Subscriptions,11.480000,0.520000,12.000000,0.290000,0.230000
Total,,,489.680000,21.320000,511.000000,11.500000,9.820000



    ID: 2
    Number of Payments: 142
    Date of Payout: 12-Feb-2024
    Payout Amount: £2429.51
    


,Section,Payment Schedule,net_amount,total_fees,gross_amount,gocardless_fees,app_fees
0,Beavers,Activities,28.950000,1.050000,30.000000,0.460000,0.590000
1,Beavers,Monthly Subscriptions,275.520000,12.480000,288.000000,6.960000,5.520000
2,Cubs,Activities,110.950000,4.050000,115.000000,1.790000,2.260000
3,Cubs,Monthly Subscriptions,332.920000,15.080000,348.000000,8.410000,6.670000
4,Scouts,Activities,992.370000,33.630000,1026.000000,13.580000,20.050000
5,Scouts,Monthly Subscriptions,516.600000,23.400000,540.000000,13.050000,10.350000
6,Squirrels,Monthly Subscriptions,172.200000,7.800000,180.000000,4.350000,3.450000
Total,,,2429.510000,97.490000,2527.000000,48.600000,48.890000


In [407]:
import pandas as pd

# Assuming df is your DataFrame
data = {'Name': ['Alice', 'Bob', 'Charlie'],
        'Age': [25, 30, 35],
        'Salary': [50000, 60000, 75000]}

df = pd.DataFrame(data)

# Calculate the sum of each column
column_sums = df.sum()

# Create a new DataFrame with the sums as a new row
total_row = pd.DataFrame([column_sums.values], columns=column_sums.index, index=['Total'])

# Concatenate the original DataFrame with the total row
df_with_total = pd.concat([df, total_row])

# Display the DataFrame with the total row
print(df_with_total)


                  Name  Age  Salary
0                Alice   25   50000
1                  Bob   30   60000
2              Charlie   35   75000
Total  AliceBobCharlie   90  185000
